In [1]:
import math
import numpy
import pandas as pd

## General Properties

In [2]:
mechanical_eff = 0.99
# Air Properties
gamma_air = 1.4
gamma_g = 1.33333
c_p_air = 1.005
c_p_gas = 1.148
R = 287

## Define Ranges

Define the range for the stage loading coefficient, flow coefficient and the reaction. Using the obtained values, calculate the other values.

Use the following data:
1. Stage Loading Coefficient -> 2.5 - 5.0
2. Reaction -> 0.45 - 0.49
3. Flow Coefficient -> 0.3 - 1.0
4. Exit Absolute Mach Number -> 0.3 - 0.55

In [3]:
range_controller = 10
range_flow_coefficient = numpy.linspace(0.5, 1.0, 25)
range_degree_reaction = numpy.linspace(0.4, 0.5, range_controller)
range_alpha_3 = numpy.linspace(10, 40, 30)
range_exit_velocity = numpy.linspace(0.3 * numpy.sqrt(gamma_g*R*1020.985), 0.55 * numpy.sqrt(gamma_g*R*1020.985), 10)

#### Calculation Exit Axial Velocity

In [4]:
def calc_exit_axial_velocity(range_alpha_3, range_exit_velocity):
    c_a_5 = range_exit_velocity * (numpy.cos(numpy.radians(range_alpha_3)))
    return c_a_5
    
data_exit = []

for i in range_alpha_3:
    for j in range_exit_velocity:
        c_a_5 = calc_exit_axial_velocity(i, j)
        data_exit.append((i, j, c_a_5))

df_exit = pd.DataFrame(data_exit, columns = ['alpha_3','c_5','c_a_5'])
# extract data from df for further computations
c_a_5 = df_exit['c_a_5']
alpha_3 = df_exit['alpha_3']
df_exit.to_csv('_outputs/output_velocities.csv', index=False)

### CHECK FILTER
1. Remove the values of U based on the maximum hub speed.

2. Remove the values of the stage loading based on industry practices.

In [5]:
data_U_psi = []

for i in c_a_5:
    for j in range_flow_coefficient:
        U = i / j
        stage_loading = 2*216000/(U**2)
        if 0 <= U <= 335 and 0.7 <= j <= 1.0 and stage_loading <= 10:
            data_U_psi.append((i, j, U, stage_loading))
 
df_U_psi = pd.DataFrame(data_U_psi, columns = ['c_a_5','flow_coefficient','U', 'stage_loading'])
df_U_psi.to_csv('_outputs/output_U.csv', index=False)

# extract data from df for further computations
c_a_5 = df_U_psi['c_a_5']
flow_coefficient = df_U_psi['flow_coefficient']
stage_loading = df_U_psi['stage_loading']

## Computation of results
Use of dataframe to organize and analyze the data. 

Results saved as .csv file -> "_outputs/output.csv"

In [6]:
def calc_reaction(alpha, psi, phi):
    """
    This function calculates the degree of reaction.
    Input: Swirl angle "alpha", flow coefficient "phi", blade loading coefficient "psi"
    Output: Returns the value of the degree of coefficient.
    """
    tan_beta_3 = numpy.tan(numpy.radians(alpha)) + phi**(-1)
    reaction = (tan_beta_3 * 2 * phi - 0.5 * psi) / 2
    return reaction

"""
data = []

for i in alpha_3:
    for j in stage_loading:
        for k in flow_coefficient:
            if 0.7 <= k <= 0.9: 
                result = calc_reaction(i, j, k)
                if 0.35 <= result <= 0.65:
                    data.append((i, j, k, result))
            
df = pd.DataFrame(data, columns=['alpha_3', 'stage_loading', 'flow_coefficient', 'reaction'])
df.to_csv('_outputs/output.csv', index=False)
"""


"\ndata = []\n\nfor i in alpha_3:\n    for j in stage_loading:\n        for k in flow_coefficient:\n            if 0.7 <= k <= 0.9: \n                result = calc_reaction(i, j, k)\n                if 0.35 <= result <= 0.65:\n                    data.append((i, j, k, result))\n            \ndf = pd.DataFrame(data, columns=['alpha_3', 'stage_loading', 'flow_coefficient', 'reaction'])\ndf.to_csv('_outputs/output.csv', index=False)\n"